In [1]:
import pandas as pd

# Charger le fichier CSV
file_path = "full_graph.csv"  # Remplacez par le chemin correct si nécessaire
df = pd.read_csv(file_path)

# Ajouter une colonne "known" initialisée à 0 pour chaque nœud
df["known"] = 0

# Sauvegarder le fichier modifié (optionnel)
df.to_csv("full_graph_updated.csv", index=False)

# Afficher un aperçu des données
print(df.head())


   _id   _labels                          name  _start  _end _type  known
0  0.0  :Concept        n_gram_language_models     NaN   NaN   NaN      0
1  1.0  :Concept          linear_approximation     NaN   NaN   NaN      0
2  2.0  :Concept             linear_regression     NaN   NaN   NaN      0
3  3.0  :Concept                        vector     NaN   NaN   NaN      0
4  4.0  :Concept  independent_random_variables     NaN   NaN   NaN      0
